<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Split Monopole `GiRaFFEfood` Initial Data for `GiRaFFE`

## Author: Patrick Nelson

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Split_Monopole.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Split_Monopole.py)

**Notebook Status:** <font color='green'><b> In-Progress </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation1). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

## Introduction: 
We need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; here, we will only be implementing the "Split Monopole" initial data, given by Table 3 in [the original paper](https://arxiv.org/pdf/1704.00599.pdf). The vector potential is
\begin{align}
A_r &= -\frac{aC}{8}\cos\theta \left( 1 + \frac{4M}{r} \right), \\
A_\theta &= 0, \\
A_\phi &= M^2 C [1-\cos \theta + a^2 f(r) \cos \theta \sin^2 \theta],
\end{align}
and the electric field is
\begin{align}
E_r &= -\frac{C a^3}{8\alpha M^3} f'(r) \cos \theta \sin^2 \theta \\
E_\theta &= -\frac{Ca}{8\alpha}[\sin \theta + a^2 f(r) \sin \theta (2 \cos^2 \theta-\sin^2 \theta) ] - \beta^r \sqrt{\gamma} \frac{a C}{8 r^2}\left( 1+\frac{4M}{r}\right) \\
E_\phi &= \frac{\beta^r}{\alpha M} Ca^2 f'(r) \cos \theta \sin^2 \theta,
\end{align}
where
\begin{align}
f(r) =& \ \frac{r^2(2r-3M)}{8M^3} L \left(\frac{2M}{r}\right) \\
&+ \frac{M^2+3Mr-6r^2}{12M^2} \ln \frac{r}{2M} \\
&+ \frac{11}{72} + \frac{M}{3r} + \frac{r}{2M} - \frac{r^2}{M^2}, \\
L(x) =& \ {\rm Li}(x) + \frac{1}{2} \ln x \ln (1-x).
\end{align}
The function $L(x)$ is known as the dilogarithm function, and 
$$ {\rm Li}(x) = -\int_{0}^{1} \frac{\ln(1-tx)}{t} dt = \sum_{k=1}^{\infty} \frac{x^k}{k^2}. $$
Now, to use this initial data scheme, we need to transform the above into the quantities actually tracked by `GiRaFFE` and HydroBase: $A_i$, $B^i$, $\tilde{S}_i$, $v^i$, and $\Phi$. Of these quantities, `GiRaFFEfood` will only set $A_i$, $v^i$, and $\Phi=0$, then call a separate function to calculate  $\tilde{S}_i$; `GiRaFFE` itself will call a function to set $B^i$ before the time-evolution begins. This can be done with eqs. 16 and 18, here given in that same order:
\begin{align}
v^i &= \alpha \frac{\epsilon^{ijk} E_j B_k}{B^2} -\beta^i \\
B^i &= \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k \\
\end{align}
In the simulations, $B^i$ will be calculated numerically from $A_i$; however, it will be useful to analytically calculate $B^i$ to use calculating the initial $v^i$.

This module requires the use of the NRPy+ [Shifted Kerr-Schild initial data module](../Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb)

In [ ]:
def Ar_SM(x,y,z, **params):


In [ ]:
def Ath_SM(x,y,z, **params):


In [ ]:
def Aph_SM(x,y,z, **params):


In [ ]:
def ValenciavU_SM(**params):
